# Testing Graphical User Interfaces

In this chapter, we explore how to generate tests for Graphical User Interfaces (GUIs), abstracting from our [previous examples on Web testing](WebFuzzer.ipynb).  Building on general means to extract user interface elements and to activate them, our techniques generalize to arbitrary graphical user interfaces, from rich Web applications to mobile apps.

**Prerequisites**

* We build on the Web server introduced in the [chapter on Web testing](WebFuzzer.ipynb).

## Automated GUI Interaction

With our Web server: no JavaScript, no rich interfaces.  Also: Limited to Web.

How can we automate interaction?

### Our Web Server, Again

We (again) run our Web server.

In [ ]:
import fuzzingbook_utils

In [ ]:
from WebFuzzer import init_db, start_httpd, webbrowser, print_httpd_messages, print_url, ORDERS_DB

In [ ]:
db = init_db()

In [ ]:
httpd_process, httpd_url = start_httpd()
print_url(httpd_url)

In [ ]:
from IPython.core.display import HTML, display, Image

In [ ]:
HTML(webbrowser(httpd_url))

### Remote Control with Selenium

Let us just look at the GUI, above.  We do not assume we can access the HTML source, or even the URL of the current page.  All we assume is that there is a set of *user interface elements* we can interact with.

Documentation is available [here.](https://selenium-python.readthedocs.io/index.html)

In [ ]:
from selenium import webdriver

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
from selenium.webdriver.firefox.options import Options

In [ ]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [ ]:
# Alternative: Chrome

# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('window-size=700x230')
# driver = webdriver.Chrome(options=options)

In [ ]:
driver.get(httpd_url)

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

### Filling out Forms

In [ ]:
name = driver.find_element_by_name("name")
name.send_keys("Jane Doe")

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
email = driver.find_element_by_name("email")
email.send_keys("j.doe@example.com")

In [ ]:
city = driver.find_element_by_name('city')
city.send_keys("Seattle")

In [ ]:
zip = driver.find_element_by_name('zip')
zip.send_keys("98104")

In [ ]:
terms = driver.find_element_by_name('terms')
terms.click()

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
submit = driver.find_element_by_name('submit')
submit.click()

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

### Navigating

In [ ]:
driver.back()

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
links = driver.find_elements_by_tag_name("a")

In [ ]:
links[0].get_attribute('href')

In [ ]:
links[0].click()

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
driver.back()

In [ ]:
print_httpd_messages()

In [ ]:
Image(driver.get_screenshot_as_png())

## Retrieving UI Elements

In [ ]:
driver.get(httpd_url)

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
ui_elements = driver.find_elements_by_tag_name("input")

In [ ]:
for element in ui_elements:
    print(element.get_attribute('name'), element.get_attribute('type'), element.text)

In [ ]:
ui_elements = driver.find_elements_by_tag_name("button")

In [ ]:
for element in ui_elements:
    print(element.get_attribute('name'), element.get_attribute('type'), element.text)

\todo{Use a grammar to fill out a set of fields, generating a sequence of calls:}

```python
name.click()
city.send_keys("foo")
```

In [ ]:
class UIElement(object):
    def __init__(self, name):
        self._name = name
    def name(self):
        return self._name

    def send_keys(self, s):
        pass
    def click(self):
        pass

In [ ]:
class FillUIElement(UIElement):
    def __repr__(self):
        return "fill(%s)" % repr(self.name())

In [ ]:
class ClickUIElement(UIElement):
    def __repr__(self):
        return "click(%s)" % repr(self.name())

In [ ]:
def ui_elements(driver):
    elements = set()
    for elem in driver.find_elements_by_tag_name("input"):
        input_type = elem.get_attribute("type")
        input_name = elem.get_attribute("name")
        if input_name is None:
            input_name = elem.text

        if input_type in ["button", "checkbox", "radio"]:
            elements.add(ClickUIElement(input_name))
        elif input_type in ["text", "number", "password"]:
            elements.add(FillUIElement(input_name))
        else:
            # TODO: Handle more types here
            elements.add(FillUIElement(input_name))
    
    for elem in driver.find_elements_by_tag_name("button"):
        button_type = elem.get_attribute("type")
        button_name = elem.get_attribute("name")
        if button_name is None:
            button_name = elem.text
        if button_type != "reset":
            elements.add(ClickUIElement(button_name))

    for elem in driver.find_elements_by_tag_name("a"):
        a_href = elem.get_attribute("href")
        if a_href is not None:
            elements.add(ClickUIElement(elem.text))
    
    return elements

In [ ]:
ui_elements(driver)

This set of interactive elements makes up a _page_.

## Systematic GUI Exploration

\todo{Fill out forms, click on all links, exploring one page after another}

\todo{Create a finite state automaton}

\todo{Convert it into a grammar and traverse all paths}

## Fun with FuzzingBook

\todo{Create a full map of fuzzingbook.org, only by navigating}

In [ ]:
driver.get("https://www.fuzzingbook.org/")

In [ ]:
Image(driver.get_screenshot_as_png())

In [ ]:
links = driver.find_elements_by_tag_name("a")
for link in links:
    print(link.tag_name, link.text, link.get_attribute("href"))

That's it – we're done!

In [ ]:
driver.quit()

In [ ]:
httpd_process.terminate()

In [ ]:
import os

In [ ]:
for temp_file in [ORDERS_DB, "geckodriver.log", "ghostdriver.log"]:
    if os.path.exists(temp_file):
        os.remove(temp_file)

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

_Cite relevant works in the literature and put them into context, as in:_

The idea of ensuring that each expansion in the grammar is used at least once goes back to Burkhardt \cite{Burkhardt1967}, to be later rediscovered by Paul Purdom \cite{Purdom1972}.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```markdown
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_